In [3]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from rutermextract import TermExtractor
from kw_extraction_scripts import textrank

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/h.pylieva/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


In [4]:
from __future__ import division
import operator
import nltk
import string

def isPunct(word):
    return len(word) == 1 and word in string.punctuation

def isNumeric(word):
    try:
        float(word) if '.' in word else int(word)
        return True
    except ValueError:
        return False

class RakeKeywordExtractor:

    def __init__(self):
        self.stopwords = set(nltk.corpus.stopwords.words('russian'))
        self.top_fraction = 1 #consider top third candidate keywords by score

    def _generate_candidate_keywords(self, sentences):
        phrase_list = []
        for sentence in sentences:
            words = map(lambda x: "|" if x in self.stopwords else x,
                nltk.word_tokenize(sentence.lower()))
            phrase = []
            for word in words:
                if word == "|" or isPunct(word):
                    if len(phrase) > 0:
                        phrase_list.append(phrase)
                        phrase = []
                else:
                    phrase.append(word)
        return phrase_list

    def _calculate_word_scores(self, phrase_list):
        word_freq = nltk.FreqDist()
        word_degree = nltk.FreqDist()
        for phrase in phrase_list:
            degree = len(list(filter(lambda x: not isNumeric(x), phrase))) - 1
            for word in phrase:
                word_freq[word] +=1
                word_degree[word] += degree
        for word in word_freq.keys():
            word_degree[word] = word_degree[word] + word_freq[word] # itself
        # word score = deg(w) / freq(w)
        word_scores = {}
        for word in word_freq.keys():
            word_scores[word] = float(word_degree[word]) / word_freq[word]
        return word_scores

    def _calculate_phrase_scores(self, phrase_list, word_scores):
        phrase_scores = {}
        for phrase in phrase_list:
            phrase_score = 0
            for word in phrase:
                phrase_score += word_scores[word]
                phrase_scores[" ".join(phrase)] = phrase_score
        return phrase_scores
    
    def extract(self, text, incl_scores=False):
        sentences = nltk.sent_tokenize(text)
        phrase_list = self._generate_candidate_keywords(sentences)
        word_scores = self._calculate_word_scores(phrase_list)
        phrase_scores = self._calculate_phrase_scores(
            phrase_list, word_scores)
        sorted_phrase_scores = sorted(phrase_scores.items(),
            key=operator.itemgetter(1), reverse=True)
        n_phrases = len(sorted_phrase_scores)
        if incl_scores:
            return sorted_phrase_scores[0:int(float(n_phrases)/self.top_fraction)]
        else:
            return map(lambda x: x[0],
                sorted_phrase_scores[0:int(float(n_phrases)/self.top_fraction)])

In [5]:
ALPHA_RU = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяіїґ'
ALPHA_EN = 'abcdefghijklmnopqrstuvwxwz'
LETTERS_LIST = list(ALPHA_RU + ALPHA_EN)
term_extractor = TermExtractor()
rake = RakeKeywordExtractor()

In [67]:
prod = pd.read_csv('data/3000_products.csv')
desc = pd.read_csv('data/3000_desc.csv')
df = prod.set_index('id').join(desc.set_index('product_id'))
df = df.dropna(subset=['description'])

In [7]:
def tokenize_description(desc, algorithm = 'rutermextractor'):
#     tokens_list = []
    if not str(desc)=='nan' and len(desc)>0:
            if algorithm == 'rutermextractor':
                terms = term_extractor(desc) 
#                 for term in terms:
                tokens_list = [term.normalized for term in terms]
            elif algorithm == 'rake':
                tokens_list = list(rake.extract(desc, incl_scores=False))
            elif algorithm == 'textrank':
                tokens_list = textrank(text)
    return tokens_list

In [69]:
df.description =(df.description
                       .astype('str')
                       .apply(lambda x: re.sub(r"\s+", " ", BeautifulSoup(x, "lxml").text.lower()))
                       .apply(lambda x: ' '.join(word for word in x.split() if word not in (LETTERS_LIST))))
df['desc_tokens'] = df.description.apply(lambda x: tokenize_description(x))

In [70]:
df['desc_tokens_rake'] = df.description.apply(lambda x: tokenize_description(x, 'rake'))

In [71]:
df[:10]

,name,url,category_id,cat_caption,description,desc_tokens,desc_tokens_rake
id,,,,,,,
3446225,"Шина 15,5 R38 (400R965) Ф-2А Белшина",https://prom.ua/p3446225-product.html,801222,шины,"шина 15,5 r38 ф-2а белшина приминение :трактор...","[задние ведущие колёса трактора, тракторы, зад...",[камерное тип рисунка протектора повышенной пр...
21252332,"Платье ""Cоло"" мол. — Модель 689м (замена круже...",https://prom.ua/p21252332-product.html,35402,платья женские,товар высокого качества по ценам от производит...,"[разный возраст комплекции, последние тенденци...",[-коллекция одежды постоянно обновляется расши...
21652889,Мелки цветные (10 цветов),https://prom.ua/p21652889-product.html,35402,платья женские,оролрлорлорлор лолдодод,"[оролрлорлорлор, лолдодод]",[]
21665429,Бамбуковый чехол для iPhone 4/4S с птицей,https://prom.ua/p21665429-product.html,380230,"чехлы для телефонов, mp3 плееров",бамбуковый чехол для iphone 4/4s птицей надежн...,"[айфон, наш интернет-магазин смартфонов аксесс...",[айфон 4 изображением птицы — символом скорых ...
21665585,Бамбуковый чехол для iPhone 4/4S с кельтским о...,https://prom.ua/p21665585-product.html,380230,"чехлы для телефонов, mp3 плееров",бамбуковый чехол для iphone 4/4s кельтским орн...,"[iphone 4/4s, наш интернет-магазин смартфонов ...",[гравировке деревянных чехлов предлагает наш и...
21665756,Чехол для iPhone 4/4S Статуя Свободы,https://prom.ua/p21665756-product.html,380230,"чехлы для телефонов, mp3 плееров",чехол для iphone 4/4s статуя свободы данная мо...,"[айфон, такая деревянный кейс, срок службы сма...",[iphone 4/4s статуя свободы данная модель чехл...
21665947,Бамбуковый чехол для iPhone 4/4S Цветок,https://prom.ua/p21665947-product.html,380230,"чехлы для телефонов, mp3 плееров",бамбуковый чехол для iphone 4/4s цветок защитн...,"[iphone 4/4s, чехол, дизайн, айфон, особый сво...",[чехол украшен очень красивым цветком нетленны...
21666081,Бамбуковый чехол для iPhone 4/4S Грампластинка,https://prom.ua/p21666081-product.html,380230,"чехлы для телефонов, mp3 плееров",бамбуковый чехол для iphone 4/4s грампластинка...,"[такая чехол, бамбуковый чехол, бамбук, специа...",[благодаря своим природным свойствам летом дер...
21666313,Бамбуковый чехол для iPhone 4/4S с нотами,https://prom.ua/p21666313-product.html,380230,"чехлы для телефонов, mp3 плееров",бамбуковый чехол для iphone 4/4s нотами купить...,"[ноты, чистый защитный чехол, просмотр фильмов...",[любом другом городе украины стало намного про...


In [72]:
df.drop_duplicates(subset=['name'])[:1000].to_csv('prods_with_tokenized_desc.csv',sep='\t', index=True)

## Test on separate phrases

In [17]:
text = 'Душевая стойка из нержавеющей стали с гидромассажем Okyanus'
text = 'услуги адвоката по уголовным делам'
text = 'услуга адвокат уголовный дело'
text = 'купить лилового и синего слона с большими ушами'

In [18]:
display(tokenize_description(text, algorithm='rake'))

display(tokenize_description(text, algorithm='textrank'))

display(tokenize_description(text, algorithm='rutermextractor'))

['синего слона', 'купить лилового']

['лиловый', 'синий слон']

['синий слон', 'большие уши']

In [19]:
term_extractor('text')

In [20]:
a = [1]
a.extend([4,2])

In [24]:
def list_diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

In [25]:
A = [1,2,3,4]
B = [2,5]

In [27]:
diff(B, A)

[5]